In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [2]:
df = pd.read_csv(r'funda_train.csv', encoding='utf-8-sig')
print(df)

FileNotFoundError: [Errno 2] No such file or directory: 'funda_train.csv'

In [ ]:
pd.set_option('display.max_columns', 15)
df.head()

In [ ]:
df0 = df.fillna(0)
df0

In [ ]:
# transacted_date, transacted_time 데이터 타입 string으로 인식하여 datetime64로 변환(column.dt.시간단위를 통해서 원하는 시간 부분을 추출해서 활용할 수 있음)
df0['transacted_date'] = pd.to_datetime(df0['transacted_date'])
df0['transacted_time'] = pd.to_datetime(df0['transacted_time'])

In [ ]:
# 2016년 이전 데이터 확인
year1970 = (df0['transacted_date'] < '2016-06-01')

In [ ]:
filtered_df=df0.loc[year1970]

In [ ]:
filtered_df

In [ ]:
from datetime import *

In [ ]:
#일자별 매출 그룹바이
daily_amount = df0.groupby(['store_id','transacted_date'])['amount'].sum()
daily_amount

In [ ]:
daily_amount = daily_amount.reset_index()

In [ ]:
#인덱스를 거래일자로 재설정
daily_amount.index = daily_amount['transacted_date']
daily_amount.set_index('transacted_date', inplace=True)
daily_amount.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 한글폰트 오류 해결

from matplotlib import font_manager, rc
font_path = "./malgun.ttf"#폰트파일 위치
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

plt.style.use('seaborn-poster')#스타일 서식 지정
plt.rcParams['axes.unicode_minus']=False #마이너스 부호 출력 지정

In [ ]:
#일자별 매출 시각화
daily_amount.plot()
plt.show()

## ARIMA 모델 ##

### 전처리 ###

In [ ]:
from pylab import rcParams
import statsmodels.api as sm
import itertools
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
# 월별 판매 예측
salesbymonth = daily_amount.amount.resample('M').sum()
salesbymonth

In [ ]:
salesbymonth = salesbymonth.reset_index()

In [ ]:
#인덱스를 거래월자로 재설정
salesbymonth.index = salesbymonth['transacted_date']
salesbymonth.set_index('transacted_date', inplace=True)
salesbymonth.head()

In [ ]:
salesbymonth.plot()

In [ ]:
# stationary 확인

timeseries = salesbymonth['amount']
timeseries.rolling(12).mean().plot()#12개월평균

In [ ]:
timeseries.rolling(12).std().plot()#12개월분산

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
decomposition = seasonal_decompose(salesbymonth['amount'], model='additive', 
                            extrapolate_trend='freq')

In [ ]:
fig = plt.figure()
fig = decomposition.plot()
fig.set_size_inches(15,7)

## stationary 검증 방법 ##

In [ ]:
from statsmodels.tsa.stattools import adfuller

In [ ]:
result = adfuller(salesbymonth['amount'])

In [ ]:
result
#  0.12033314079559654-> p값 0.05보다 작아야 stationary -> 즉 stationary가 아니다

###    - 검증 조건 ( p-value : 5% 이내면 reject으로 대체가설 선택됨 )
     -> 귀무가설(H0): non-stationary.
     -> 대체가설 (H1): stationary. 

In [ ]:
# stationary check 기능 함수
def adf_check(ts):
    result = adfuller(ts)
    if result[1] <=0.05:
        print('Stationary{}'.format(result[1]))
    else:
        print('Non-stationary {}'.format(result[1]))

In [ ]:
adf_check(salesbymonth['amount'])

## differencing ##

In [ ]:
salesbymonth['1st diff'] = salesbymonth['amount'] - salesbymonth['amount'].shift(1)

In [ ]:
salesbymonth.head()

In [ ]:
adf_check(salesbymonth['1st diff'].dropna())

In [ ]:
salesbymonth['1st diff'].plot()

## 2차 differencing ##

In [ ]:
salesbymonth['2nd diff'] = salesbymonth['1st diff'] - salesbymonth['1st diff'].shift(1)

In [ ]:
salesbymonth['2nd diff'].plot()

In [ ]:
adf_check(salesbymonth['2nd diff'].dropna())

## 3차 differencing ##

In [ ]:
salesbymonth['3rd diff'] = salesbymonth['2nd diff'] - salesbymonth['2nd diff'].shift(1)

### 1차 differencing보다 더 작아짐 1st, 2nd 둘다 사용 가능 ###

In [ ]:
salesbymonth['seasonal diff'] = salesbymonth['amount'] - salesbymonth['amount'].shift(12)

In [ ]:
salesbymonth['seasonal diff'].plot()

In [ ]:
adf_check(salesbymonth['seasonal diff'].dropna())

In [ ]:
# 1st differencing에 seasonal differencing을 하는 것도 방법임

In [ ]:
salesbymonth['seasonal 1st diff'] = salesbymonth['1st diff'] - salesbymonth['1st diff'].shift(12)

In [ ]:
salesbymonth['seasonal 1st diff'].plot()

In [ ]:
adf_check(salesbymonth['seasonal 1st diff'].dropna())

In [ ]:
# 한번 더 differencing 하기 
salesbymonth['seasonal 2nd diff'] = salesbymonth['2nd diff'] - salesbymonth['2nd diff'].shift(12)

In [ ]:
adf_check(salesbymonth['seasonal 2nd diff'].dropna())
# 해당 데이터 쓰면 됨

In [ ]:
# 3차 seasonal differencing 하기 
salesbymonth['seasonal 3rd diff'] = salesbymonth['3rd diff'] - salesbymonth['3rd diff'].shift(12)

In [ ]:
adf_check(salesbymonth['seasonal 3rd diff'].dropna())

In [ ]:
# d값은 2차 differencing이므로 2를 씀
d = 2, D = 2

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
plot_acf(salesbymonth['1st diff'].dropna());

In [ ]:
plot_acf(salesbymonth['2nd diff'].dropna());

In [ ]:
plot_acf(salesbymonth['seasonal 2nd diff'].dropna());

In [ ]:
plot_pacf(salesbymonth['seasonal 2nd diff'].dropna(), method='ywm')#partial AutoCorrelation

In [ ]:
# acf에서 음수 1개씩 나왔으므로 p, q값은 각 P=1. Q=1씀

In [ ]:
plot_pacf(salesbymonth['1st diff'].dropna(), method='ywm')#partial AutoCorrelation

### 2017년 3월 2018년 2월까지 주기성이 보임, 점차적으로 증가하다 2018에서는 유지에서 머무르고 있다 ###

### 파라미터 찾기 ###

이제 본격적으로 ARIMA 모델을 만들어야 한다. ARIMA 모델을 만들기 위해 p, d, q 값을 지정해 주어야 하는데 일반적으로 p가 0이면 MA 모델을 따르고, q가 0이면 AR 모델을 따른다고 한다. d가 0이면 정상성을 보유한 모델이라고 보는데, 발생확률이 변하지 않는 ARMA 모델이라고 간주하면 된다고 한다. 우린 아직 이 데이터가 어떤 모형을 따르는 지 모르기 때문에 최적의 결과를 가져오는 p,d,q 값을 찾아야한다. 그 방법은 크게 두가지로 임의의 값들을 다 넣어 테스트 해보거나, pmdarima 패키지의 auto_arima 함수로 찾아내면 된다. 

In [ ]:
#방법 1. p,d,q의 조합을 만들어 하나하나 ARIMA 모델을 돌려봄

#p,d,q 범위 생성 0~2
p = d = q = range(0, 2)

import itertools#반복되는 요소를 이어붙이는 모듈
pdqa = list(itertools.product(p, d, q))

seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
#x의 0~2범위 요소를 하나씩 꺼내서 pdq를 리스트안에 붙여서 for 반복문 돌리기

for param in pdqa:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(salesbymonth_train, order=param, seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)                                
            results = mod.fit()
            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
!pip install pmdarima

In [ ]:
#방법 2. auto_arima 함수로 자동 추출
from pmdarima import auto_arima
stepwise_model = auto_arima(salesbymonth['amount'], start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

## 하이퍼 파라미터 정하기 ##

p=0, d=1, q=0



### AIC(Akaike Information Criterion) 값이 제일 작은 조합을 선택 ###

### 학습 및 검증 ###

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

In [ ]:
model = ARIMA(salesbymonth['amount'], order=(0,1,0))

In [ ]:
result= model.fit(trend='c', full_output=True, disp=True)

In [ ]:
result.summary()
#AIC	1296.617 모델 성능 의미 

In [ ]:
result.resid.plot()

In [ ]:
result.resid.plot(kind='kde')
# 가우스분포 0이므로 모델 피팅이 어느정도 된 것

## 예측 그래프 그려보기 ##

In [ ]:
result.plot_predict()

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt
from pandas import datetime

## 3개월 후 예측 ##

In [ ]:
#modelfitting
model = ARIMA(train, order=(0, 1, 0))  
fitted = model.fit(disp=-1)  

In [ ]:
# Forecast
fc, se, conf  = fitted.forecast(steps=3, alpha=0.05)
# 예측값, stderr, upper bound, lower bound 

In [ ]:
# Make as pandas series
fc_series = pd.Series(fc, index=[79,80,81,82,83,84,85,86,87,88,89,90,91,92,93])
fc_series

In [ ]:
# plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(train, label='training')
plt.plot(test, label='actual')
plt.plot(fc_series, label='forecast')

plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)

plt.title('Forecast')
plt.legend(loc='upper right', fontsize=8)
plt.show()

In [ ]:
# Accuracy metrics
def forecast_accuracy(forecast, actual):
    mape = np.mean(np.abs(forecast - actual)/np.abs(actual))  # MAPE
    me = np.mean(forecast - actual)             # ME
    mae = np.mean(np.abs(forecast - actual))    # MAE
    mpe = np.mean((forecast - actual)/actual)   # MPE
    rmse = np.mean((forecast - actual)**2)**.5  # RMSE
    corr = np.corrcoef(forecast, actual)[0,1]   # corr
    mins = np.amin(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    maxs = np.amax(np.hstack([forecast[:,None], 
                              actual[:,None]]), axis=1)
    minmax = 1 - np.mean(mins/maxs)             # minmax
    acf1 = acf(fc-test)[1]                      # ACF1
    return({'mape':mape, 'me':me, 'mae': mae, 
            'mpe': mpe, 'rmse':rmse, 'acf1':acf1, 
            'corr':corr, 'minmax':minmax})

forecast_accuracy(fc, test.values)

#> {'mape': 0.02250131357314834,
#>  'me': 3.230783108990054,
#>  'mae': 4.548322194530069,
#>  'mpe': 0.016421001932706705,
#>  'rmse': 6.373238534601827,
#>  'acf1': 0.5105506325288692,
#>  'corr': 0.9674576513924394,
#>  'minmax': 0.02163154777672227}